# Lab #1 (Part 1): Ames 주택 가격 예측 실습 과제

이 과제에서는 오늘 배운 `Pipeline`, `ColumnTransformer`, 그리고 다양한 회귀 모델을 종합적으로 활용하여 Ames Housing 데이터셋의 주택 가격을 예측하는 전체 머신러닝 워크플로를 경험합니다.

### 🎯 학습 목표

1.  다양한 타입의 특성(수치형, 범주형)을 식별하고 적절한 전처리 전략을 수립할 수 있다.
2.  `ColumnTransformer`와 `Pipeline`을 사용하여 전처리 및 모델링 과정을 하나의 워크플로로 묶을 수 있다.
3.  선형 회귀와 다항 회귀 모델을 구축하고, 그 성능을 비교/분석할 수 있다.
4.  모델의 예측 성능을 R-squared와 RMSE 지표로 평가하고 해석할 수 있다.

---

### 📜 Step 1: 데이터 로드 및 기본 탐색

Kaggle의 Ames Housing 데이터셋을 불러옵니다. 이 데이터셋은 실제 주택 판매 데이터를 담고 있으며, 매우 다양한 특성을 포함하고 있습니다. 여기서는 실습의 편의를 위해 일부 특성만 선택하여 사용합니다.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
# 데이터셋 URL

housing = fetch_openml(name="house_prices", as_frame=True)
df = housing.data.join(housing.target)

In [2]:

# 실습에 사용할 특성 선택
use_cols = [
    'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'TotalBsmtSF', 'GrLivArea', # 수치형
    'MSZoning', 'Street', 'BldgType', 'HouseStyle', 'Neighborhood', # 범주형
    'SalePrice' # 타겟
]
df = df[use_cols]

# 데이터 정보 확인 및 결측치 확인
print("데이터 정보:")
df.info()
print("\n결측치 개수:")
print(df.isnull().sum())

df.head()

데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LotArea       1460 non-null   int64 
 1   OverallQual   1460 non-null   int64 
 2   OverallCond   1460 non-null   int64 
 3   YearBuilt     1460 non-null   int64 
 4   TotalBsmtSF   1460 non-null   int64 
 5   GrLivArea     1460 non-null   int64 
 6   MSZoning      1460 non-null   object
 7   Street        1460 non-null   object
 8   BldgType      1460 non-null   object
 9   HouseStyle    1460 non-null   object
 10  Neighborhood  1460 non-null   object
 11  SalePrice     1460 non-null   int64 
dtypes: int64(7), object(5)
memory usage: 137.0+ KB

결측치 개수:
LotArea         0
OverallQual     0
OverallCond     0
YearBuilt       0
TotalBsmtSF     0
GrLivArea       0
MSZoning        0
Street          0
BldgType        0
HouseStyle      0
Neighborhood    0
SalePrice       0
dtype: int64


,LotArea,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,MSZoning,Street,BldgType,HouseStyle,Neighborhood,SalePrice
0,8450,7,5,2003,856,1710,RL,Pave,1Fam,2Story,CollgCr,208500
1,9600,6,8,1976,1262,1262,RL,Pave,1Fam,1Story,Veenker,181500
2,11250,7,5,2001,920,1786,RL,Pave,1Fam,2Story,CollgCr,223500
3,9550,7,5,1915,756,1717,RL,Pave,1Fam,2Story,Crawfor,140000
4,14260,8,5,2000,1145,2198,RL,Pave,1Fam,2Story,NoRidge,250000


---

### 📜 Step 2: 데이터 분할 및 특성 정의

모델 학습을 위해 데이터를 훈련 세트와 테스트 세트로 분할합니다. 그 후, 파이프라인에 적용할 수치형 특성과 범주형 특성의 목록을 정의합니다.

In [ ]:
from sklearn.model_selection import train_test_split

# 독립 변수(X)와 종속 변수(y) 분리
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# 훈련/테스트 데이터 분할 (8:2 비율, 재현성을 위해 random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 특성 목록 정의
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

print("수치형 특성:", numeric_features)
print("범주형 특성:", categorical_features)

---

### 📜 Step 3: 전처리 파이프라인 구축

`ColumnTransformer`를 사용하여 수치형과 범주형 특성에 각기 다른 전처리 파이프라인을 구축하세요.

- **수치형 특성**: 결측치를 중앙값(`median`)으로 대체하고, `StandardScaler`를 사용하여 표준화하세요.
- **범주형 특성**: 결측치를 최빈값(`most_frequent`)으로 대체하고, `OneHotEncoder`를 사용하여 인코딩하세요. (`handle_unknown='ignore'` 옵션 추가)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# 수치형 데이터 전처리 파이프라인
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # TO-DO: 알맞은 전략 채우기
    ('scaler', StandardScaler()) # TO-DO: 스케일러 채우기
])

# 범주형 데이터 전처리 파이프라인
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # TO-DO: 알맞은 전략 채우기
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # TO-DO: 인코더 채우기
])

# ColumnTransformer로 전처리기 통합
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

---

### 📜 Step 4: 모델 파이프라인 생성 및 학습/평가

이제 위에서 만든 `preprocessor`와 모델을 결합하여 두 개의 최종 파이프라인을 만듭니다.

1.  `LinearRegression` 모델을 사용하는 파이프라인
2.  `PolynomialFeatures` (degree=2)와 `LinearRegression`을 사용하는 다항 회귀 파이프라인

각 파이프라인을 학습시키고, 테스트 데이터에 대한 **RMSE**와 **R-squared** 값을 계산하여 출력하세요.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

# ----- 모델 1: 선형 회귀 파이프라인 ----- #

# TO-DO: preprocessor와 LinearRegression을 결합한 파이프라인 생성
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# 학습
lr_pipeline.fit(X_train, y_train)

# 예측 및 평가
y_pred_lr = lr_pipeline.predict(X_test)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

print("[선형 회귀 모델]")
print(f"Test RMSE: ${rmse_lr:,.2f}")
print(f"Test R-squared: {r2_lr:.4f}")


# ----- 모델 2: 다항 회귀 파이프라인 ----- #

# TO-DO: preprocessor, PolynomialFeatures, LinearRegression을 결합한 파이프라인 생성
poly_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('regressor', LinearRegression())
])

# 학습
poly_pipeline.fit(X_train, y_train)

# 예측 및 평가
y_pred_poly = poly_pipeline.predict(X_test)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))
r2_poly = r2_score(y_test, y_pred_poly)

print("\n[다항 회귀(Degree=2) 모델]")
print(f"Test RMSE: ${rmse_poly:,.2f}")
print(f"Test R-squared: {r2_poly:.4f}")

---

### 📜 Step 5: 결과 비교 및 해석

두 모델의 성능 지표(RMSE, R-squared)를 비교하고, 어떤 모델이 이 데이터셋에 더 적합한지 자신의 생각을 간략하게 작성해 보세요. 왜 그런 결과가 나왔을지, 다항 회귀 모델의 장단점은 무엇일지 고민해 보세요.

**[TODO: 여기에 분석 결과를 작성하세요]**

...

### 🌟 (심화) 추가적으로 시도해볼 것

- 다항 회귀의 차수(degree)를 3으로 변경했을 때 결과는 어떻게 달라지나요?
- `StandardScaler` 대신 `MinMaxScaler`를 사용하면 최종 성능에 변화가 있을까요? (선형 모델에서는 큰 차이가 없을 수 있습니다. 이유를 생각해 보세요!)
- 사용한 특성 외에 다른 특성들을 추가하여 모델 성능을 더 높여보세요.